In [1]:
import pandas as pd

In [2]:
df1 = pd.DataFrame([{'userid':'u1', 'first_name':'user1', 'last_name':'test', 'title':'manager'}
                    , {'userid':'u2', 'first_name':'user2', 'last_name':'test', 'title':'creator'}
                    , {'userid':'u4', 'first_name':'user4', 'last_name':'test', 'title':'Analysts'}])
df1

,first_name,last_name,title,userid
0,user1,test,manager,u1
1,user2,test,creator,u2
2,user4,test,Analysts,u4


In [3]:
df2 = pd.DataFrame([{'userid':'u1', 'first_name':'user1', 'last_name':'test', 'title':'designer'}
                    , {'userid':'u2', 'first_name':'user2', 'last_name':'test', 'title':'creator'}

                    , {'userid':'u3', 'first_name':'user2', 'last_name':'test', 'title':'creator'}])
df2

,first_name,last_name,title,userid
0,user1,test,designer,u1
1,user2,test,creator,u2
2,user2,test,creator,u3


In [4]:
mergedDF = pd.merge(df1, df2,how='outer',on=['userid','first_name','last_name','title'],indicator=True)
mergedDF

,first_name,last_name,title,userid,_merge
0,user1,test,manager,u1,left_only
1,user2,test,creator,u2,both
2,user4,test,Analysts,u4,left_only
3,user1,test,designer,u1,right_only
4,user2,test,creator,u3,right_only


__Proposed Solution: Convert '_merge' indicator column into a situation specific descriptive column:__

* left_only / right_only indicates a change if the userid, first_name, last_name match.
    * Keep left value
    

* both, nochange

* left_only ( no matching user id ) is deleted

* right_only ( no matching user id) is new


In [5]:
# This distinction should be considered carefully or data may be processed inconsistently given a large sample:
# Why keep title 'manager' from df1? This action is implying df2 is older? 

grp_cols = ['userid', 'first_name', 'last_name']
describers = {'_merge':{'left_only':'changed', 'right_only':'changed'}}

step1DF = mergedDF.groupby(grp_cols).filter(lambda x: x.title.count() > 1)[mergedDF.groupby(grp_cols).filter(lambda x: x.title.count() > 1)['_merge'] == 'left_only']
step1DF['_merge'].astype('object') # convert from categorical back to object
step1DF['_merge'] = 'changed'

mergedDF.groupby(grp_cols).filter(lambda x: x.title.count() > 1)

,first_name,last_name,title,userid,_merge
0,user1,test,manager,u1,left_only
3,user1,test,designer,u1,right_only


In [6]:
# This distinction should be considered carefully or dataframe will be inconsistent with large sample:
# Why is u4 considered to be new and u3 deleted? This action implies df2 is newer? 
# 

describers = {'_merge':{'both': 'Nochange', 'left_only':'New', 'right_only':'Deleted'}}
step2DF = mergedDF.groupby(grp_cols).filter(lambda x: x.title.count() == 1).replace(describers)
mergedDF.groupby(grp_cols).filter(lambda x: x.title.count() == 1)

,first_name,last_name,title,userid,_merge
1,user2,test,creator,u2,both
2,user4,test,Analysts,u4,left_only
4,user2,test,creator,u3,right_only


In [7]:
pd.concat([step1DF, step2DF])

,first_name,last_name,title,userid,_merge
0,user1,test,manager,u1,changed
1,user2,test,creator,u2,Nochange
2,user4,test,Analysts,u4,New
4,user2,test,creator,u3,Deleted
